In [15]:
import numpy as np
import os
os.environ['pRT_input_data_path'] = "/home/natalie/.local/lib/python3.8/site-packages/petitRADTRANS/input_data_std/input_data"
from target import Target
from retrieval import Retrieval
from parameters import Parameters
import figures as figs

def init_retrieval(brown_dwarf='2M0355',PT_type='PTgrad',chem='freechem',
                   Nlive=400,tol=0.5,cloud_mode='gray',GP=True):

    brown_dwarf = Target(brown_dwarf)
    output=f'{chem}_{PT_type}_N{Nlive}_ev{tol}' # output folder name

    constant_params={} # add if needed
    free_params = {'rv': ([2,20],r'$v_{\rm rad}$'),
                'vsini': ([0,40],r'$v$ sin$i$'),
                'log_g':([3,5],r'log $g$'),
                'epsilon_limb': [(0.2,1), r'$\epsilon_\mathrm{limb}$']} # limb-darkening coefficient (0-1)

    if PT_type=='PTknot':
        pt_params={'T0' : ([1000,4000], r'$T_0$'), # bottom of the atmosphere (hotter)
                'T1' : ([0,4000], r'$T_1$'),
                'T2' : ([0,4000], r'$T_2$'),
                'T3' : ([0,4000], r'$T_3$'),
                'T4' : ([0,4000], r'$T_4$'),} # top of atmosphere (cooler)
        free_params.update(pt_params)

    if PT_type=='PTgrad':
        pt_params={'dlnT_dlnP_0': ([0.,0.4], r'$\nabla T_0$'), # gradient at T0 
                'dlnT_dlnP_1': ([0.,0.4], r'$\nabla T_1$'), 
                'dlnT_dlnP_2': ([0.,0.4], r'$\nabla T_2$'), 
                'dlnT_dlnP_3': ([0.,0.4], r'$\nabla T_3$'), 
                'dlnT_dlnP_4': ([0.,0.4], r'$\nabla T_4$'), 
                'T0': ([1000,4000], r'$T_0$')} # at bottom of atmosphere
        free_params.update(pt_params)

    # if equilibrium chemistry, define [Fe/H], C/O, and isotopologue ratios
    if chem=='equchem':
        chemistry={'C/O':([0,1], r'C/O'), 
                'Fe/H': ([-1.5,1.5], r'[Fe/H]'), 
                'log_C12_13_ratio': ([1,12], r'log $\mathrm{^{12}C/^{13}C}$'), 
                'log_O16_18_ratio': ([1,12], r'log $\mathrm{^{16}O/^{18}O}$'), 
                'log_O16_17_ratio': ([1,12], r'log $\mathrm{^{16}O/^{17}O}$')}
            
    if chem=='quequchem': # quenched equilibrium chemistry
        chemistry={'C/O':([0,1], r'C/O'), 
                'Fe/H': ([-1.5,1.5], r'[Fe/H]'), 
                'log_C12_13_ratio': ([1,12], r'log $\mathrm{^{12}C/^{13}C}$'), 
                'log_O16_18_ratio': ([1,12], r'log $\mathrm{^{16}O/^{18}O}$'), 
                'log_O16_17_ratio': ([1,12], r'log $\mathrm{^{16}O/^{17}O}$'),
                'log_Pqu_CO_CH4': ([-6,2], r'log P$_{qu}$(CO,CH$_4$,H$_2$O)'),
                'log_Pqu_NH3': ([-6,2], r'log P$_{qu}$(NH$_3$)'),
                'log_Pqu_HCN': ([-6,2], r'log P$_{qu}$(HCN)')}  
        
    # if free chemistry, define VMRs
    if chem=='freechem': 
        chemistry={'log_H2O':([-12,-1],r'log H$_2$O'),
                'log_12CO':([-12,-1],r'log $^{12}$CO'),
                'log_13CO':([-12,-1],r'log $^{13}$CO'),
                'log_C18O':([-12,-1],r'log C$^{18}$O'),
                'log_C17O':([-12,-1],r'log C$^{17}$O'),
                'log_CH4':([-12,-1],r'log CH$_4$'),
                'log_NH3':([-12,-1],r'log NH$_3$'),
                'log_HCN':([-12,-1],r'log HCN'),
                'log_HF':([-12,-1],r'log HF'),
                'log_H2(18)O':([-12,-1],r'log H$_2^{18}$O'),
                'log_H2S':([-12,-1],r'log H$_2$S')}
        
    if cloud_mode=='gray':
        cloud_props={'log_opa_base_gray': ([-10,3], r'log $\kappa_{\mathrm{cl},0}$'),  
                    'log_P_base_gray': ([-6,3], r'log $P_{\mathrm{cl},0}$'), # pressure of gray cloud deck
                    'fsed_gray': ([0,20], r'$f_\mathrm{sed}$')} # sedimentation parameter for particles
        free_params.update(cloud_props)

    if cloud_mode=='MgSiO3':
        cloud_props={'fsed': ([0,20], r'$f_\mathrm{sed}$'), # sedimentation parameter for particles
                    'sigma_lnorm': ([0.8,1.5], r'$\sigma_{l,norm}$'), # width of the log-normal particle distribution
                    'log_Kzz':([5,15],r'log $K_{zz}$')} # eddy diffusion parameter (atmospheric mixing)
        free_params.update(cloud_props)
        
    if GP==True: # add uncertainty scaling
        GP_params={'log_a': ([-1,1], r'$\log\ a$'), # one is enough, will be multipled with order/det error
                'log_l': ([-3,0], r'$\log\ l$')}
        free_params.update(GP_params)

    free_params.update(chemistry)
    parameters = Parameters(free_params, constant_params)
    cube = np.random.rand(parameters.n_params)
    parameters(cube)

    retrieval=Retrieval(target=brown_dwarf,parameters=parameters,
                    output_name=output,chemistry=chem,PT_type=PT_type)
    return retrieval

#retrieval=init_retrieval(brown_dwarf='2M1425',PT_type='PTgrad',chem='freechem',Nlive=300,tol=0.5)
retrieval=init_retrieval(brown_dwarf='2M0355',PT_type='PTgrad',chem='freechem',Nlive=400,tol=0.5)
#retrieval.PMN_lnL()
#only_params=['rv','vsini','log_g','T0','log_H2O','log_12CO','log_13CO','log_HF','log_H2(18)O','log_H2S']
#retrieval.evaluate(only_params=only_params,split_corner=False)
#retrieval.evaluate(makefigs=True)
#retrieval.run_retrieval(N_live_points=400,evidence_tolerance=0.5,molecules=['13CO'],bayes=False)

retrieval2=init_retrieval(brown_dwarf='2M1425',PT_type='PTgrad',chem='freechem',Nlive=400,tol=0.5)
#retrieval2.evaluate(makefigs=False)
#figs.VMR_plot(retrieval,retrieval_object2=retrieval2)

/home/natalie/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [ ]:
import pathlib
import pickle
import numpy as np

path=pathlib.Path('/home/natalie/Desktop/atm_retrieval/2M0355/freechem_PTgrad_N400_ev0.5/params_dict.pickle')
with open(path,'rb') as file:
    final_params=pickle.load(file)
retrieval.params_dict=final_params
retrieval.cross_correlation('13CO')

{'SNR_13CO': 12.504598284455263}

In [16]:
path=pathlib.Path('/home/natalie/Desktop/atm_retrieval/2M1425/freechem_PTgrad_N400_ev0.5/params_dict.pickle')
with open(path,'rb') as file:
    final_params2=pickle.load(file)
retrieval2.params_dict=final_params2
retrieval2.cross_correlation('13CO')

figs.compare_two_CCFs(retrieval,retrieval2,'13CO')

AttributeError: 'Retrieval' object has no attribute 'CCF_list'

# Clouds

In [ ]:
# shouldn't opacity below cloud be high, to block all emission from deeper layers?
# if it's zero, there is effectively no cloud

import numpy as np

params={'log_P_base_gray':-4,'log_opa_base_gray':1,'fsed_gray':4}
wave_micron=np.linspace(1,3,2)
pressure = np.logspace(-6,2,5)
opa_gray_cloud = np.zeros((len(wave_micron),len(pressure))) # gray cloud = independent of wavelength
opa_gray_cloud[:,pressure>10**(params['log_P_base_gray'])] = 10**(params['log_opa_base_gray']) # [bar] constant below cloud base
# Opacity decreases with power-law above the base
above_clouds = (pressure<=10**(params['log_P_base_gray']))
opa_gray_cloud[:,above_clouds]=(10**(params['log_opa_base_gray']))*(pressure[above_clouds]/10**(params['log_P_base_gray']))**params['fsed_gray']

print(opa_gray_cloud)
print(pressure)

[[1.e-07 1.e+01 1.e+01 1.e+01 1.e+01]
 [1.e-07 1.e+01 1.e+01 1.e+01 1.e+01]]
[1.e-06 1.e-04 1.e-02 1.e+00 1.e+02]


# Table for paper

In [10]:
import pathlib
import pickle
import numpy as np

path=pathlib.Path('/home/natalie/Desktop/atm_retrieval/2M0355/freechem_PTgrad_N400_ev0.5/params_dict.pickle')
with open(path,'rb') as file:
    final_params=pickle.load(file)

path=pathlib.Path('/home/natalie/Desktop/atm_retrieval/2M1425/freechem_PTgrad_N400_ev0.5/params_dict.pickle')
with open(path,'rb') as file:
    final_params2=pickle.load(file)

In [11]:
dec=3
table=["C/O &","$[$C/H$]$ &","log $^{12}$CO/$^{13}$CO &","log $^{12}$CO/C$^{17}$O &",
       "log $^{12}$CO/C$^{18}$O &","log H$_2$O/H$_2^{18}$O &"]
ratios=["C/O","C/H",'log_12CO/13CO','log_12CO/C17O','log_12CO/C18O','log_H2O/H2(18)O']

for i,r in enumerate(ratios):
      print(table[i],np.round(final_params[r][0],decimals=dec),
              "$^{+",np.round(final_params[f'{r}_err'][1][0],decimals=dec),"}_{",
              np.round(final_params[f'{r}_err'][0][0],decimals=dec),"}$ &",
              np.round(final_params2[r][0],decimals=dec),
              "$^{+",np.round(final_params2[f'{r}_err'][1][0],decimals=dec),"}_{",
              np.round(final_params2[f'{r}_err'][0][0],decimals=dec),"}$ \\\\")

C/O & 0.649 $^{+ 0.006 }_{ -0.006 }$ & 0.658 $^{+ 0.003 }_{ -0.003 }$ \\
$[$C/H$]$ & 0.58 $^{+ 0.026 }_{ -0.035 }$ & 0.395 $^{+ 0.01 }_{ -0.013 }$ \\
log $^{12}$CO/$^{13}$CO & 1.979 $^{+ 0.033 }_{ -0.031 }$ & 2.037 $^{+ 0.037 }_{ -0.036 }$ \\
log $^{12}$CO/C$^{17}$O & 6.295 $^{+ 1.504 }_{ -1.419 }$ & 5.524 $^{+ 1.387 }_{ -1.174 }$ \\
log $^{12}$CO/C$^{18}$O & 5.224 $^{+ 1.9 }_{ -1.6 }$ & 6.306 $^{+ 1.312 }_{ -1.396 }$ \\
log H$_2$O/H$_2^{18}$O & 4.311 $^{+ 3.109 }_{ -1.487 }$ & 2.708 $^{+ 3.052 }_{ -0.204 }$ \\


In [12]:
import pathlib
import pickle
import numpy as np

path=pathlib.Path('/home/natalie/Desktop/atm_retrieval/2M0355/freechem_PTgrad_N400_ev0.5/evidence_dict.pickle')
with open(path,'rb') as file:
    evidence=pickle.load(file)

path=pathlib.Path('/home/natalie/Desktop/atm_retrieval/2M1425/freechem_PTgrad_N400_ev0.5/evidence_dict.pickle')
with open(path,'rb') as file:
    evidence2=pickle.load(file)

dec=2
table=["$^{13}$CO",'HF','H$_2$S','H$_2^{18}$O','CH$_4$']
molec=['13CO','HF','H2S','H2(18)O', 'CH4']

for i,m in enumerate(molec):
      print(table[i],'&',
            np.round(final_params[f'SNR_{m}'],decimals=dec),"&",
            np.round(evidence[f'sigma_{m}'],decimals=dec),"&",
            np.round(final_params2[f'SNR_{m}'],decimals=dec),"&",
            np.round(evidence2[f'sigma_{m}'],decimals=dec),"\\\\")

$^{13}$CO & 12.44 & 13.41 & 3.54 & 7.98 \\
HF & 16.87 & 11.64 & 5.46 & 15.81 \\
H$_2$S & 6.21 & 4.63 & -0.53 & 9.28 \\
H$_2^{18}$O & 2.58 & 1.14 & 1.47 & 3.05 \\
CH$_4$ & -0.67 & 2.2 & 4.89 & 5.49 \\


In [4]:
dec=3

table=["$v_{\\text{rad}}$ [km/s] & Radial velocity & $\mathcal{U}$(2,20) &",
       "$v\\text{sin}i$ [km/s] & Projected rotational velocity & $\mathcal{U}$(0,40) &",
       "log $g$ [cm/s$^2$] & Surface gravity & $\mathcal{U}$(3,5) &",
       "$\epsilon_\mathrm{limb}$ & Limb-darkening coefficient & $\mathcal{U}$(0.2,1) &",
       "",
       "log H$_2$O & log$_{10}$ VMR of H$_2$O & $\mathcal{U}$(-12,-1) & ",
       "log $^{12}$CO & log$_{10}$ VMR of $^{12}$CO & $\mathcal{U}$(-12,-1) &",
       "log $^{13}$CO & log$_{10}$ VMR of $^{13}$CO & $\mathcal{U}$(-12,-1) & ",
       "log C$^{18}$O & log$_{10}$ VMR of C$^{18}$O & $\mathcal{U}$(-12,-1) &",
       "log C$^{17}$O & log$_{10}$ VMR of C$^{17}$O & $\mathcal{U}$(-12,-1) &",
       "log CH$_4$ & log$_{10}$ VMR of CH$_4$ & $\mathcal{U}$(-12,-1) &",
       "log NH$_3$ & log$_{10}$ VMR of NH$_3$ & $\mathcal{U}$(-12,-1) & ",
       "log HCN & log$_{10}$ VMR of HCN & $\mathcal{U}$(-12,-1) &",
       "log HF & log$_{10}$ VMR of HF & $\mathcal{U}$(-12,-1) & ",
       "log H$_2$S & log$_{10}$ VMR of H$_2$S & $\mathcal{U}$(-12,-1) & ",
       "log H$_2^{18}$O & log$_{10}$ VMR of H$_2^{18}$O & $\mathcal{U}$(-12,-1) &",
      "",
      "$T_0$ [K] & Temperature at $P_0=10^2\,$bar& $\mathcal{U}$(1000,4000) &",
      " $\\nabla T_0$ & Temperature gradient at $P_0=10^2\,$bar& $\mathcal{U}$(0,0.4) &",
      "$\\nabla T_1$ & Temperature gradient at $P_1=10^0\,$bar& $\mathcal{U}$(0,0.4) &",
      "$\\nabla T_2$ & Temperature gradient at $P_2=10^0\,$bar& $\mathcal{U}$(0,0.4) &",
      "$\\nabla T_3$ & Temperature gradient at $P_3=10^0\,$bar& $\mathcal{U}$(0,0.4) &",
      "$\\nabla T_4$ & Temperature gradient at $P_4=10^0\,$bar& $\mathcal{U}$(0,0.4) &",
      "",
      "log $\kappa_{\mathrm{cl},0}$ [cm$^2$/g] & Opacity at cloud base & $\mathcal{U}$(-10,3) & ",
      "log $P_{\mathrm{cl},0}$ [bar] & Cloud base pressure & $\mathcal{U}$(-6,3) &",
      "$f_\mathrm{sed}$ & Cloud decay power & $\mathcal{U}$(0,20) &",
      "",
      "log $a$ & GP amplitude & $\mathcal{U}$(-1,1) &",
      "log $l$ [nm] & GP length-scale & $\mathcal{U}$(-3,0) &"]
table_vals=['rv','vsini','log_g','epsilon_limb',"",'log_H2O',"log_12CO","log_13CO",
            "log_C18O","log_C17O","log_CH4","log_NH3","log_HCN","log_HF","log_H2S",
            "log_H2(18)O","",'T0','dlnT_dlnP_0','dlnT_dlnP_1','dlnT_dlnP_2',
            'dlnT_dlnP_3','dlnT_dlnP_4',"","log_opa_base_gray","log_P_base_gray",
            "fsed_gray","","log_a","log_l"]

if True:
      for i,tpar in enumerate(table_vals):
            if tpar=="":
                  print("\hline")
            else:
                  print(table[i],np.round(final_params[tpar],decimals=dec),
                        "$^{+",np.round(final_params[f'{tpar}_err'][1],decimals=dec),"}_{",
                        np.round(final_params[f'{tpar}_err'][0],decimals=dec),"}$ &",
                        np.round(final_params2[tpar],decimals=dec),
                        "$^{+",np.round(final_params2[f'{tpar}_err'][1],decimals=dec),"}_{",
                        np.round(final_params2[f'{tpar}_err'][0],decimals=dec),"}$ \\\\")

$v_{\text{rad}}$ [km/s] & Radial velocity & $\mathcal{U}$(2,20) & 13.251 $^{+ 0.009 }_{ -0.009 }$ & 5.538 $^{+ 0.045 }_{ -0.046 }$ \\
$v\text{sin}i$ [km/s] & Projected rotational velocity & $\mathcal{U}$(0,40) & 3.043 $^{+ 0.081 }_{ -0.081 }$ & 31.575 $^{+ 0.186 }_{ -0.232 }$ \\
log $g$ [cm/s$^2$] & Surface gravity & $\mathcal{U}$(3,5) & 4.753 $^{+ 0.034 }_{ -0.034 }$ & 4.975 $^{+ 0.01 }_{ -0.011 }$ \\
$\epsilon_\mathrm{limb}$ & Limb-darkening coefficient & $\mathcal{U}$(0.2,1) & 0.721 $^{+ 0.141 }_{ -0.155 }$ & 0.741 $^{+ 0.036 }_{ -0.048 }$ \\
\hline
log H$_2$O & log$_{10}$ VMR of H$_2$O & $\mathcal{U}$(-12,-1) &  -2.999 $^{+ 0.026 }_{ -0.031 }$ & -3.197 $^{+ 0.008 }_{ -0.01 }$ \\
log $^{12}$CO & log$_{10}$ VMR of $^{12}$CO & $\mathcal{U}$(-12,-1) & -2.735 $^{+ 0.026 }_{ -0.035 }$ & -2.92 $^{+ 0.01 }_{ -0.013 }$ \\
log $^{13}$CO & log$_{10}$ VMR of $^{13}$CO & $\mathcal{U}$(-12,-1) &  -4.717 $^{+ 0.04 }_{ -0.044 }$ & -4.958 $^{+ 0.037 }_{ -0.039 }$ \\
log C$^{18}$O & log$_{10}$ VMR o

# Make plots for presentation

In [ ]:
import getpass
import os
import numpy as np
import matplotlib.pyplot as plt
os.environ['OMP_NUM_THREADS'] = '1' # to avoid using too many CPUs

if getpass.getuser() == "grasser": # when runnig from LEM
    os.environ['pRT_input_data_path'] ="/net/lem/data2/pRT_input_data"
    os.environ['OMP_NUM_THREADS'] = '1' # important for MPI
    from mpi4py import MPI 
    comm = MPI.COMM_WORLD # important for MPI
    rank = comm.Get_rank() # important for MPI
    from atm_retrieval.target import Target
    from atm_retrieval.likelihood import Retrieval
    from atm_retrieval.parameters import Parameters
    import matplotlib
    matplotlib.use('Agg') # disable interactive plotting
elif getpass.getuser() == "natalie": # when testing from my laptop
    os.environ['pRT_input_data_path'] = "/home/natalie/.local/lib/python3.8/site-packages/petitRADTRANS/input_data_std/input_data"
    from target import Target
    from likelihood import Retrieval
    from parameters import Parameters
    from pRT_model import pRT_spectrum


M0355 = Target('2M0355')
data_wave,data_flux,data_err=M0355.load_spectrum()

constant_params = {'vsini': 2, # rotational velocity
                'rv': 11.92,
                'log_Kzz': 7.5, # eddy diffusion parameter (atmospheric mixing)
                'fsed': 2, # sedimentation parameter for particles
                'P_base_gray': 1, # pressure of gray cloud deck
                'fsed_gray': 2,
                'opa_base_gray': 0.8, # opacity of gray cloud deck
                'sigma_lnorm': 1.05, # width of the log-normal particle distribution of MgSiO3
                'log_MgSiO3' : 0, # scaling wrt chem equilibrium, 0 = equilibrium abundance 
                'log_H2O': -2.9,#-2.9,
                'log_12CO': -2.67,#-2.66,
                'log_13CO':-4.9,
                'log_C18O':-12, #-8.3,
                'log_C17O':-12,  #-8.7,
                'log_CH4':-12, #-8.6,
                'log_NH3':-12, #-8.9,
                'log_HCN':-12, #-8,
                'T1' : 2500, # bottom of the atmosphere (hotter)
                'T2' : 1500,
                'T3' : 1270,
                'T4' : 313, # top of atmosphere (cooler)
                'log_g':4.95,
                } 

free_params = {}
parameters = Parameters(free_params,constant_params)
params=parameters.params


output='2M0355_test4'
retrieval=Retrieval(target=M0355,parameters=parameters,output_name=output)
atmosphere_objects=retrieval.atmosphere_objects

species=retrieval.get_species(param_dict=parameters.params)
model_flux=pRT_spectrum(parameters=params,data_wave=data_wave,target=M0355,
                        atmosphere_objects=atmosphere_objects,species=species,
                        free_chem=True).make_spectrum()


In [ ]:
params_H2O=params.copy()
params_H2O['log_12CO']=-12
params_H2O['log_13CO']=-12
H2O_flux=pRT_spectrum(parameters=params_H2O,data_wave=data_wave,target=M0355,
                        atmosphere_objects=atmosphere_objects,species=species,
                        free_chem=True).make_spectrum()

params_CO=params.copy()
params_CO['log_H2O']=-12
params_CO['log_13CO']=-12
CO_flux=pRT_spectrum(parameters=params_CO,data_wave=data_wave,target=M0355,
                        atmosphere_objects=atmosphere_objects,species=species,
                        free_chem=True).make_spectrum()

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(7,3),dpi=200,sharex=True)
order=5
det=0
for i in range(2):
    ax[i].plot(data_wave[order,det],data_flux[order,det],lw=1,alpha=0.8,c='k',label='data')
    ax[i].set_xlim(np.nanmin(data_wave[order,det]),np.nanmax(data_wave[order,det]))
    ax[i].yaxis.set_visible(False) # remove ylabels because anyway unitless

ax[0].plot(data_wave[order,det],model_flux[order,det],lw=1,alpha=0.8,c='c',label='model')
ax[1].plot(data_wave[order,det],H2O_flux[order,det],lw=1,alpha=0.8,c='tab:blue',label='H$_2$O')
ax[1].plot(data_wave[order,det],CO_flux[order,det],lw=1,alpha=0.8,c='tab:orange',label='CO')

ax[0].legend(fontsize=8)
ax[1].legend(fontsize=8)
ax[1].set_xlabel('Wavelength [nm]')
fig.tight_layout(h_pad=-0.1)
fig.savefig('/home/natalie/Desktop/PhD/SupJup/plots/spectrum.jpg')

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(7,2),dpi=200,sharex=True)
order=5
det=0
#ax.plot(data_wave[order,det],data_flux[order,det],lw=1,alpha=0.8,c='k',label='data')
ax.set_xlim(np.nanmin(data_wave[order,det]),np.nanmax(data_wave[order,det]))
ax.plot(data_wave[order,det],model_flux[order,det],lw=1,alpha=0.8,c='c',label='model')
#ax.legend(fontsize=8)
ax.yaxis.set_visible(False) # remove ylabels because anyway unitless
ax.set_xlabel('Wavelength [nm]')
fig.tight_layout()
fig.savefig('/home/natalie/Desktop/PhD/SupJup/plots/model.jpg')

In [ ]:
params_bad=params.copy()
params_bad['log_H2O']=-3.5
params_bad['log_CO']=-2
params_bad['vsini']=7
params_bad['T1']=1700
bad_flux=pRT_spectrum(parameters=params_bad,data_wave=data_wave,target=M0355,
                        atmosphere_objects=atmosphere_objects,species=species,
                        free_chem=True).make_spectrum()
fig,ax=plt.subplots(1,1,figsize=(7,2),dpi=200,sharex=True)
order=5
det=0
#ax.plot(data_wave[order,det],data_flux[order,det],lw=1,alpha=0.8,c='k',label='data')
ax.set_xlim(np.nanmin(data_wave[order,det]),np.nanmax(data_wave[order,det]))
ax.plot(data_wave[order,det],bad_flux[order,det],lw=1,alpha=1,c='c',label='model')
#ax.legend(fontsize=8)
ax.yaxis.set_visible(False) # remove ylabels because anyway unitless
ax.set_xlabel('Wavelength [nm]')
fig.tight_layout()
fig.savefig('/home/natalie/Desktop/PhD/SupJup/plots/bad_model1.jpg')